In [1]:
## This notebook contains modularized code for our two-step approach
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans

In [2]:
#### Data Preprocessing ####
# read netFlow data
# data1 = pd.read_csv('~/Desktop/netFlow-02-11-2017.csv', sep = ',')
# data_collab = data1[(((data1.srcAddr == '128.143.2.75') & (data1.srcPort == 443)) | ((data1.dstAddr =='128.143.2.75') & (data1.dstPort == 443)))]

#### First step is to get anomaly scores from LOF/IForest, append to dataset.
# input a dataframe, an anomaly detection method.
# LOF, IForest, or nothing.
def labeling(df, method):
    # first step is to generate a list of anomaly scores using chosen method.
    #if(method == "IForest"):
#         train = test_sample1_pages[['bytes','pkts','dur','rate']]
#         test = test_sample2_pages[['bytes','pkts','dur','rate']]
#         clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
#         clf.fit(train)
    if(method == "LOF"):
        clf = LocalOutlierFactor(n_neighbors=20)
        y_pred = clf.fit_predict(df)
        df['LOF_label'] = y_pred
    return(df)
# take 50% sample data, then label it. 
# data_collab_labeled = labeling(data_collab.sample(frac = 0.5, replace = False), "LOF") 


#### Second step is to apply K-means clustering to the same dataset 
# kmeans = KMeans(n_clusters=553, random_state=0).fit(sample) # select K based on sqrt(n/2)
# result = kmeans.labels_
# data_collab_labeled['cluster_label'] = result
# save the result..
# data_collab_labeled.to_csv("~/Desktop/collab_clean_sample.csv")

# Read in sample data generated by extracint collab-related netFlow traffic.
Collab_sample = pd.read_csv('~/Desktop/collab_clean_sample.csv')[["bytes",'pkts','dur','rate','LOF_label','cluster_label']]
Collab_sample.head(5)

,bytes,pkts,dur,rate,LOF_label,cluster_label
0,1936,14,0.074,209297.63,1,299
1,1500,14,0.065,184615.90,1,438
2,3972,16,0.270,117688.89,1,73
3,2294,14,0.169,108591.72,1,73
4,13030,24,0.100,1042400.99,1,78


In [3]:
len(Collab_sample[Collab_sample.LOF_label == -1]) / len(Collab_sample)

0.10000130492511361

In [5]:
#### Third step is to make use of the result of step 1 & 2, to reduce misclassified normal instances(reduce false alarm).
# threshold is the maximum percentage of anomalies allowed in df

def normal_profiling(df, threshold):
    clusterID = df.cluster_label.values
    df['normal_profile_labels'] = df['LOF_label']
    labels = df["cluster_label"].value_counts().index.tolist()
    # when the proportion of anomalies is larger than a threshold
    while (len(df[df.normal_profile_labels == -1]) / len(df) >= threshold):
        # set anything that has label x, yet is classified by LOF to be an anomaly to be equal to 1. 
        x = labels.pop(0)
        df.loc[((df['cluster_label'] == x) & (df['normal_profile_labels'] == -1)),'normal_profile_labels'] = 1
    return(df)
# case2: apply K-means repeatedly <-- not realistic because time-complexity
Collab_norm = normal_profiling(Collab_sample, 0.05)
len(Collab_norm[Collab_norm.normal_profile_labels == -1]) / len(Collab_norm)

0.04823166335541919

In [6]:
#### Forth step is to use majority vote to get final results.
def maj_vote(df):
    list = []
    pool = set(df[df.normal_profile_labels == -1].cluster_label) # a set of IDs of all clusters that contain anomalies
    # do majority vote, and return a list of anomalous clusters
    for i in pool:
        # for each of cluster i, calculate proportion of anomaly
        proportion_anomaly = len(df[(df['cluster_label'] == i) & (df['normal_profile_labels'] == -1)]) / len(df[df.cluster_label == i])
        if proportion_anomaly > 0.5:
            list.extend([i])
    return(list)
anomaly_clusters = maj_vote(Collab_norm)
len(anomaly_clusters)

102

In [7]:
# different sized culster for different algos for different datasets.

## step1 data subsetting
# 3 domains: collab, ECE, Shanti pages

## port 443 


## step2 
# 3 combination of algos

## step3
# 